[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LjBLincoln/Machine_Learning/blob/master/tensorflow.ipynb)

#Hello TensorFlow

In [0]:
import tensorflow as tf
import os

print("******************************")
print("     Hello TensorFlow !!!     ")
print("******************************")


input("press any key to continue.")

print("1.     Constant       ")

a = tf.constant(3.0, dtype=tf.float32,name="Constant_a")

b = tf.constant(4.0) # also tf.float32 implicitly

total = a + b

print(a)
print(b)
print(total)

print(tf.Session().run(total))

input("press any key to continue.")

print("2.      Variable       ")

my_variable = tf.get_variable("my_variable", [1, 2, 3])

my_int_variable = tf.get_variable("my_int_variable", [1, 2, 3], dtype=tf.int32,
  initializer=tf.zeros_initializer)

other_variable = tf.get_variable("other_variable", dtype=tf.int32,
  initializer=tf.constant([23, 42]))


print(my_variable)
print(my_int_variable)
print(other_variable)



variable_a = tf.get_variable("variable_a",[2,3], dtype=tf.int32)
variable_b = tf.get_variable("variable_b",[2,3], dtype=tf.int32)

print("variable_a :",variable_a)

print("variable_b :",variable_b)

with tf.Session() as sess:
    sess.run(total)
    file_writer = tf.summary.FileWriter('/tmp/logs', sess.graph)
 


# TensorBoard

In [0]:
import tensorflow as tf
import os

with tf.name_scope("AddExample"):

    a = tf.Variable(3.0, name="a")

    b = tf.Variable(4.0, name="b") # also tf.float32 implicitly
 
    c = tf.add(a, b, name ='add')
    
    print(c)

with tf.Session() as sess:
    #sess.run(tf.global_variables_initializer())
    #sess.run(c)
    file_writer = tf.summary.FileWriter('/tmp/logs', sess.graph)
 

Tensor("AddExample/add:0", shape=(), dtype=float32)


# TensorBoard Placeholder

In [0]:
import tensorflow as tf 
import numpy as np 

with tf.name_scope("PlaceholderExample"):
    #定义形状为[2,2]的单精度浮点数矩阵
    x = tf.placeholder(tf.float32, shape = (2 , 2 ), name = "x")
    y = tf.matmul(x, x, name = "matmul")
	
with tf.Session() as sess :
    rand_array = np.random.rand(2, 2)
    print(sess.run(y, feed_dict = { x: rand_array}))
    file_writer = tf.summary.FileWriter('/tmp/logs', sess.graph)

[[0.19756594 0.29739496]
 [0.29572526 0.5944579 ]]


# TensorBoard  Sparse Placeholder

In [0]:
import tensorflow as tf 
import numpy as np 

# 没有显示的指导数据形状，表示可以填充任意形状的单精度浮点数稀疏张量
x = tf.sparse_placeholder(tf.float32)
y = tf.sparse_reduce_sum(x)
	
with tf.Session() as sess :
    # 设置非零元素索引为[3, 2, 0] 和 [4, 5, 1]
    indices = np.array([[3, 2, 0], [4, 5, 1]], dtype = np.int64)
	# 设置索引为[3, 2, 0] 和 [4, 5, 1]元素的值分别为1.0和2.0
    values = np.array([1.0 , 2.0], dtype = np.float32)
    # 设置稀疏张量对应的稠密张量形状为[7, 9, 2]
    shape = np.array([7, 9, 2], dtype = np.int64)
	# 向x填充稀疏张量
    print(sess.run(y, feed_dict = { x: tf.SparseTensorValue(indices, values, shape)}))
	# 向x填充张量3元组（indices, values, shape ）
    print(sess.run(y, feed_dict = { x: (indices, values, shape)}))
    # 向x填充Numpy多为数组
    sp = tf.SparseTensor(indices = indices, values = values, dense_shape = shape)
    sp_value = sp.eval()
    print(sess.run(y, feed_dict = { x:  sp_value}))

    file_writer = tf.summary.FileWriter('/tmp/logs', sess.graph)
 


# Save ckpt to pb

In [2]:
import os, argparse
import tensorflow as tf
from tensorflow.python.framework import graph_util
 
#dir = os.path.dirname(os.path.realpath(__file__))
 
def freeze_graph(model_folder):
    # We retrieve our checkpoint fullpath
    checkpoint = tf.train.get_checkpoint_state(model_folder)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    # We precise the file fullname of our freezed graph
    absolute_model_folder = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_folder + "/frozen_model.pb"
 
    # Before exporting our graph, we need to precise what is our output node
    # this variables is plural, because you can have multiple output nodes
    output_node_names = "Accuracy/predictions"
 
    # We clear the devices, to allow TensorFlow to control on the loading where it wants operations to be calculated
    clear_devices = True
    
    # We import the meta graph and retrive a Saver
    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)
 
    # We retrieve the protobuf graph definition
    graph = tf.get_default_graph()
    input_graph_def = graph.as_graph_def()
 
    with tf.Session() as sess:
        saver.restore(sess, input_checkpoint)
 
        # We use a built-in TF helper to export variables to constant
        output_graph_def = graph_util.convert_variables_to_constants(
            sess, 
            input_graph_def, 
            output_node_names.split(",") # We split on comma for convenience
        ) 
 
        # Finally we serialize and dump the output graph to the filesystem
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))
 
 
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument("--model_folder", type=str, help="Model folder to export")
    args = parser.parse_args()
 
    freeze_graph(args.model_folder)


usage: ipykernel_launcher.py [-h] [--model_folder MODEL_FOLDER]
ipykernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-0844e656-6d4d-4a4b-81c5-a3d772b16a92.json


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


#Colab Env Test

In [12]:
! pwd
! ls -la
! cd /tmp
! pwd
! cd /home/
! pwd
! mkdir tensorflow
! cd tensorflow
! ls -la
! pwd


/content
total 20
drwxr-xr-x 1 root root 4096 Oct 17 05:11 .
drwxr-xr-x 1 root root 4096 Oct 17 02:49 ..
drwxr-xr-x 4 root root 4096 Oct 15 20:34 .config
drwxr-xr-x 2 root root 4096 Oct 15 20:47 sample_data
drwxr-xr-x 2 root root 4096 Oct 17 05:11 tensorflow
/content
/content
mkdir: cannot create directory ‘tensorflow’: File exists
total 20
drwxr-xr-x 1 root root 4096 Oct 17 05:11 .
drwxr-xr-x 1 root root 4096 Oct 17 02:49 ..
drwxr-xr-x 4 root root 4096 Oct 15 20:34 .config
drwxr-xr-x 2 root root 4096 Oct 15 20:47 sample_data
drwxr-xr-x 2 root root 4096 Oct 17 05:11 tensorflow
/content
